## 1- Customer

In [11]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'DESKTOP-GMT4735\\SQLEXPRESS'
database_name = 'Pharmacy Dataset 3.0'
sql_user_name = 'sa'
sql_password = 'Mm@12345678'

# Batch size for insert
batch_size = 100

def fetch_data_in_bulk(model, fields, uid, domain=None):
    """Fetch data from Odoo in bulk for a specific model."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain] if domain else [[]],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting customer data synchronization...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch customer data
    print("Fetching customer data...")
    customers = fetch_data_in_bulk('res.partner', ['id', 'name', 'is_company'], uid)

    if not customers:
        print("No customers fetched. Exiting.")
        return
    print(f"Fetched {len(customers)} customers.")

    # Step 3: Insert/Update into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        print("Inserting customer data into SQL Server...")
        batch_data = []

        for customer in customers:
            cust_id = customer.get('id')
            cust_name = customer.get('name')
            is_company = customer.get('is_company', False)
            cust_type = "Corporate" if is_company else "Individual"

            batch_data.append((cust_id, cust_name, cust_type))

            if len(batch_data) >= batch_size:
                for row in batch_data:
                    cursor.execute("""
                        MERGE Customers AS target
                        USING (SELECT %d AS cust_id, %s AS cust_name, %s AS cust_type) AS source
                        ON target.cust_id = source.cust_id
                        WHEN MATCHED THEN
                            UPDATE SET 
                                target.cust_name = source.cust_name,
                                target.cust_type = source.cust_type
                        WHEN NOT MATCHED THEN
                            INSERT (cust_id, cust_name, cust_type)
                            VALUES (source.cust_id, source.cust_name, source.cust_type);
                    """, row)
                conn.commit()
                batch_data = []

        # Insert remaining records
        if batch_data:
            for row in batch_data:
                cursor.execute("""
                    MERGE Customers AS target
                    USING (SELECT %d AS cust_id, %s AS cust_name, %s AS cust_type) AS source
                    ON target.cust_id = source.cust_id
                    WHEN MATCHED THEN
                        UPDATE SET 
                            target.cust_name = source.cust_name,
                            target.cust_type = source.cust_type
                    WHEN NOT MATCHED THEN
                        INSERT (cust_id, cust_name, cust_type)
                        VALUES (source.cust_id, source.cust_name, source.cust_type);
                """, row)
            conn.commit()

        print("All customer data inserted/updated successfully.")

        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the sync
process_and_insert()



Starting customer data synchronization...
Authenticated successfully. User ID: 112
Fetching customer data...
Fetched 841 customers.
Inserting customer data into SQL Server...
All customer data inserted/updated successfully.


## 2- Employees

In [13]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'DESKTOP-GMT4735\\SQLEXPRESS'
database_name = 'Pharmacy Dataset 3.0'
sql_user_name = 'sa'
sql_password = 'Mm@12345678'

# Batch size
batch_size = 100

def fetch_data_in_bulk(model, fields, uid, domain=None):
    """Fetch data from Odoo in bulk for a specific model."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain] if domain else [[]],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting employee data synchronization from POS orders...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch employee data from pos.order
    print("Fetching employee data from POS orders...")
    orders = fetch_data_in_bulk('pos.order', ['user_id'], uid)

    if not orders:
        print("No employees found in POS orders. Exiting.")
        return

    print(f"Fetched {len(orders)} POS orders.")

    # Extract unique employees
    employee_map = {}
    for order in orders:
        user = order.get('user_id')
        if user and len(user) == 2:
            emp_id = user[0]
            emp_name = user[1]
            employee_map[emp_id] = emp_name

    if not employee_map:
        print("No valid employee data found.")
        return

    print(f"Extracted {len(employee_map)} unique employees.")

    # Step 3: Insert into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        print("Inserting employee data into SQL Server...")

        batch = []
        for emp_id, emp_name in employee_map.items():
            batch.append((emp_id, emp_name))

            if len(batch) >= batch_size:
                for emp in batch:
                    cursor.execute("""
                        MERGE INTO Employees AS target
                        USING (SELECT %d AS emp_id, %s AS emp_name) AS source
                        ON target.emp_id = source.emp_id
                        WHEN MATCHED THEN
                            UPDATE SET target.emp_name = source.emp_name
                        WHEN NOT MATCHED THEN
                            INSERT (emp_id, emp_name)
                            VALUES (source.emp_id, source.emp_name);
                    """, emp)
                conn.commit()
                batch = []

        # Final batch
        if batch:
            for emp in batch:
                cursor.execute("""
                    MERGE INTO Employees AS target
                    USING (SELECT %d AS emp_id, %s AS emp_name) AS source
                    ON target.emp_id = source.emp_id
                    WHEN MATCHED THEN
                        UPDATE SET target.emp_name = source.emp_name
                    WHEN NOT MATCHED THEN
                        INSERT (emp_id, emp_name)
                        VALUES (source.emp_id, source.emp_name);
                """, emp)
            conn.commit()

        print("Employee data synchronization completed successfully.")

        cursor.close()
        conn.close()

    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the sync
process_and_insert()


Starting employee data synchronization from POS orders...
Authenticated successfully. User ID: 112
Fetching employee data from POS orders...
Fetched 275420 POS orders.
Extracted 6 unique employees.
Inserting employee data into SQL Server...
Employee data synchronization completed successfully.


## 3-Branches

In [14]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'DESKTOP-GMT4735\\SQLEXPRESS'
database_name = 'Pharmacy Dataset 3.0'
sql_user_name = 'sa'
sql_password = 'Mm@12345678'

# Batch size
batch_size = 100

def fetch_data_in_bulk(model, fields, uid, domain=None):
    """Fetch data from Odoo in bulk for a specific model."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain] if domain else [[]],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting sales team (branch) data synchronization...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch sales team data
    print("Fetching sales team data...")
    sales_teams = fetch_data_in_bulk('crm.team', ['id', 'name'], uid)

    if not sales_teams:
        print("No sales teams fetched. Exiting.")
        return

    print(f"Fetched {len(sales_teams)} sales teams.")

    # Step 3: Prepare and insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        print("Inserting branch data into SQL Server...")

        batch_data = []
        for team in sales_teams:
            branch_id = team.get('id')
            branch_name = team.get('name')
            batch_data.append((branch_id, branch_name))

            if len(batch_data) >= batch_size:
                for branch in batch_data:
                    cursor.execute("""
                        MERGE INTO Branches AS target
                        USING (SELECT %d AS branch_id, %s AS branch_name) AS source
                        ON target.branch_id = source.branch_id
                        WHEN MATCHED THEN
                            UPDATE SET target.branch_name = source.branch_name
                        WHEN NOT MATCHED THEN
                            INSERT (branch_id, branch_name)
                            VALUES (source.branch_id, source.branch_name);
                    """, branch)
                conn.commit()
                batch_data = []

        # Insert final batch
        if batch_data:
            for branch in batch_data:
                cursor.execute("""
                    MERGE INTO Branches AS target
                    USING (SELECT %d AS branch_id, %s AS branch_name) AS source
                    ON target.branch_id = source.branch_id
                    WHEN MATCHED THEN
                        UPDATE SET target.branch_name = source.branch_name
                    WHEN NOT MATCHED THEN
                        INSERT (branch_id, branch_name)
                        VALUES (source.branch_id, source.branch_name);
                """, branch)
            conn.commit()

        print("Branch data synchronization completed successfully.")

        cursor.close()
        conn.close()

    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the sync
process_and_insert()


Starting sales team (branch) data synchronization...
Authenticated successfully. User ID: 112
Fetching sales team data...
Fetched 4 sales teams.
Inserting branch data into SQL Server...
Branch data synchronization completed successfully.


## 4- Prpduct Category 

In [17]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details (Updated database name)
server_name = 'DESKTOP-GMT4735\\SQLEXPRESS'
database_name = 'Pharmacy Dataset 3.0'
sql_user_name = 'sa'
sql_password = 'Mm@12345678'

batch_size = 100

def fetch_data_in_bulk(model, fields, uid, domain=None):
    """Fetch data from Odoo in bulk for a specific model."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain] if domain else [[]],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def parse_category_path(complete_path):
    """Splits the category path into max 7 levels (main + 6 subcategories)."""
    if not complete_path:
        return [""] * 7

    parts = complete_path.split(" / ")
    parts += [""] * (7 - len(parts))  # pad if less
    return parts[:7]

def process_and_insert():
    print("Starting product category data synchronization...")

    # Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Fetch product category data
    print("Fetching product category data...")
    categories = fetch_data_in_bulk('product.category', ['id', 'complete_name'], uid)
    if not categories:
        print("No product categories fetched. Exiting.")
        return
    print(f"Fetched {len(categories)} categories.")

    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        print("Inserting product category data into SQL Server...")
        batch_data = []

        for cat in categories:
            cat_id = cat.get('id')
            parts = parse_category_path(cat.get('complete_name', ''))

            row = (cat_id,) + tuple(parts)
            batch_data.append(row)

            if len(batch_data) >= batch_size:
                for data in batch_data:
                    cursor.execute("""
                    MERGE INTO Product_Category AS target
                    USING (SELECT %d AS category_id, %s AS category_name,
                                  %s AS sub_category1, %s AS sub_category2,
                                  %s AS sub_category3, %s AS sub_category4,
                                  %s AS sub_category5, %s AS sub_category6) AS source
                    ON target.category_id = source.category_id
                    WHEN MATCHED THEN
                        UPDATE SET 
                            target.category_name = source.category_name,
                            target.sub_category1 = source.sub_category1,
                            target.sub_category2 = source.sub_category2,
                            target.sub_category3 = source.sub_category3,
                            target.sub_category4 = source.sub_category4,
                            target.sub_category5 = source.sub_category5,
                            target.sub_category6 = source.sub_category6
                    WHEN NOT MATCHED THEN
                        INSERT (category_id, category_name, sub_category1, sub_category2, sub_category3,
                                sub_category4, sub_category5, sub_category6)
                        VALUES (source.category_id, source.category_name, source.sub_category1, source.sub_category2,
                                source.sub_category3, source.sub_category4, source.sub_category5, source.sub_category6);
                    """, data)
                conn.commit()
                batch_data = []

        if batch_data:
            for data in batch_data:
                cursor.execute("""
                MERGE INTO Product_Category AS target
                USING (SELECT %d AS category_id, %s AS category_name,
                              %s AS sub_category1, %s AS sub_category2,
                              %s AS sub_category3, %s AS sub_category4,
                              %s AS sub_category5, %s AS sub_category6) AS source
                ON target.category_id = source.category_id
                WHEN MATCHED THEN
                    UPDATE SET 
                        target.category_name = source.category_name,
                        target.sub_category1 = source.sub_category1,
                        target.sub_category2 = source.sub_category2,
                        target.sub_category3 = source.sub_category3,
                        target.sub_category4 = source.sub_category4,
                        target.sub_category5 = source.sub_category5,
                        target.sub_category6 = source.sub_category6
                WHEN NOT MATCHED THEN
                    INSERT (category_id, category_name, sub_category1, sub_category2, sub_category3,
                            sub_category4, sub_category5, sub_category6)
                    VALUES (source.category_id, source.category_name, source.sub_category1, source.sub_category2,
                            source.sub_category3, source.sub_category4, source.sub_category5, source.sub_category6);
                """, data)
            conn.commit()

        print("All product category data inserted successfully.")
        cursor.close()
        conn.close()

    except Exception as e:
        print(f"SQL error: {e}")

# Run it
process_and_insert()


Starting product category data synchronization...
Authenticated successfully. User ID: 112
Fetching product category data...
Fetched 2063 categories.
Inserting product category data into SQL Server...
All product category data inserted successfully.


## 5-Products 

In [19]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'DESKTOP-GMT4735\\SQLEXPRESS'
database_name = 'Pharmacy Dataset 3.0'
sql_user_name = 'sa'
sql_password = 'Mm@12345678'

batch_size = 100
default_category_id = 10

def fetch_data_in_bulk(model, fields, uid, domain=None):
    """Fetch data from Odoo in bulk for a specific model."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain] if domain else [[]],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def fetch_product_data(uid):
    """Fetch product data from Odoo."""
    print("Fetching product data...")
    product_fields = ['id', 'name', 'categ_id', 'standard_price', 'list_price']
    products = fetch_data_in_bulk('product.product', product_fields, uid)
    if not products:
        print("No product data fetched.")
    else:
        print(f"Fetched {len(products)} products.")
    return products

def get_existing_category_ids():
    """Fetch existing category IDs from the Product_Category table in SQL Server."""
    category_ids = set()
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()
        cursor.execute("SELECT category_id FROM Product_Category")
        for row in cursor:
            category_ids.add(row[0])
        cursor.close()
        conn.close()
        print(f"Fetched {len(category_ids)} existing category IDs.")
    except pymssql.Error as e:
        print(f"SQL error fetching category IDs: {e}")
    return category_ids

def process_and_insert_products():
    print("Starting product data synchronization...")

    # Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Fetch product data
    products = fetch_product_data(uid)
    if not products:
        return

    # Fetch existing category IDs from SQL Server
    existing_category_ids = get_existing_category_ids()

    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()
        batch_data = []

        for product in products:
            product_id = product.get('id')
            product_name = product.get('name', '')
            odoo_category_id = product.get('categ_id', (False,))[0] if product.get('categ_id') else None
            product_cost = product.get('standard_price', 0.0)
            product_price = product.get('list_price', 0.0)

            sql_category_id = odoo_category_id if odoo_category_id in existing_category_ids else default_category_id

            row = (product_id, product_name, sql_category_id, str(product_cost), str(product_price))
            batch_data.append(row)

            if len(batch_data) >= batch_size:
                for data in batch_data:
                    cursor.execute("""
                        MERGE INTO Products AS target
                        USING (SELECT %d AS product_id, %s AS product_name, %d AS category_id,
                                     %s AS product_cost, %s AS product_price) AS source
                        ON target.product_id = source.product_id
                        WHEN MATCHED THEN
                            UPDATE SET
                                target.product_name = source.product_name,
                                target.category_id = source.category_id,
                                target.product_cost = source.product_cost,
                                target.product_price = source.product_price
                        WHEN NOT MATCHED THEN
                            INSERT (product_id, product_name, category_id, product_cost, product_price)
                            VALUES (source.product_id, source.product_name, source.category_id, source.product_cost, source.product_price);
                    """, data)
                conn.commit()
                batch_data = []

        if batch_data:
            for data in batch_data:
                cursor.execute("""
                    MERGE INTO Products AS target
                    USING (SELECT %d AS product_id, %s AS product_name, %d AS category_id,
                                 %s AS product_cost, %s AS product_price) AS source
                    ON target.product_id = source.product_id
                    WHEN MATCHED THEN
                        UPDATE SET
                            target.product_name = source.product_name,
                            target.category_id = source.category_id,
                            target.product_cost = source.product_cost,
                            target.product_price = source.product_price
                    WHEN NOT MATCHED THEN
                        INSERT (product_id, product_name, category_id, product_cost, product_price)
                        VALUES (source.product_id, source.product_name, source.category_id, source.product_cost, source.product_price);
                """, data)
            conn.commit()
        cursor.close()
        conn.close()
        print("Product data synchronization to Products table complete.")

    except pymssql.Error as e:
        print(f"SQL error during product processing: {e}")

# Run it
process_and_insert_products()

Starting product data synchronization...
Authenticated successfully. User ID: 112
Fetching product data...
Fetched 4459 products.
Fetched 2063 existing category IDs.
Product data synchronization to Products table complete.


## 6-Orders

In [23]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import xmlrpc.client
import pymssql
from datetime import datetime
import openpyxl

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'DESKTOP-GMT4735\\SQLEXPRESS'
database_name = 'Pharmacy Dataset 3.0'
sql_user_name = 'sa'
sql_password = 'Mm@12345678'

batch_size = 100
skipped_orders_filename = "skipped_orders_with_missing_customer.xlsx"

def fetch_data_in_bulk(model, fields, uid, domain=None):
    """Fetch data from Odoo in bulk for a specific model."""
    print(f"  * Fetching data for model '{model}'...")
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain] if domain else [[]],
            {'fields': fields}
        )
        print(f"  * Successfully fetched {len(data)} records for model '{model}'.")
        return data
    except Exception as e:
        print(f"  * Error fetching data from Odoo for model {model}: {e}")
        return []

def fetch_order_data(uid):
    """Fetch POS order data with order barcode and potential customer info."""
    print("2. Fetching POS order data from Odoo...")
    order_fields = ['id', 'order_barcode', 'date_order', 'user_id', 'partner_id', 'crm_team_id', 'amount_total', 'state']
    orders = fetch_data_in_bulk('pos.order', order_fields, uid)
    if not orders:
        print("    * No POS order data fetched from Odoo.")
    return orders

def get_existing_customer_ids():
    """Fetch existing customer IDs from the Customers table in SQL Server."""
    print("3. Fetching existing customer IDs from SQL Server...")
    customer_ids = set()
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()
        cursor.execute("SELECT cust_id FROM Customers")
        for row in cursor:
            customer_ids.add(row[0])
        cursor.close()
        conn.close()
        print(f"    * Fetched {len(customer_ids)} existing customer IDs.")
    except pymssql.Error as e:
        print(f"    * SQL error fetching customer IDs: {e}")
    return customer_ids

def write_skipped_orders_to_excel(skipped_orders, filename):
    """Writes the details of skipped orders to an Excel file."""
    if not skipped_orders:
        print("    * No skipped orders to write to Excel.")
        return

    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Skipped Orders"

    # Write headers
    headers = ["Order Barcode", "Odoo Customer ID"]
    sheet.append(headers)

    # Write data
    for order_info in skipped_orders:
        row_data = [order_info['order_barcode'], order_info['odoo_customer_id']]
        sheet.append(row_data)

    try:
        workbook.save(filename)
        print(f"    * Skipped orders details written to '{filename}'.")
    except Exception as e:
        print(f"    * Error writing skipped orders to Excel: {e}")

def process_and_insert_orders():
    print("1. Starting order data synchronization process...")

    # Authenticate with Odoo
    print("  * Attempting Odoo authentication...")
    uid = None  # Initialize uid
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("  * Odoo authentication failed. Check your credentials.")
            return
        print(f"  * Odoo authentication successful. User ID: {uid}")
    except Exception as e:
        print(f"  * Odoo authentication error: {e}")
        return

    # Fetch order data
    orders = fetch_order_data(uid)
    if not orders:
        print("Order data fetching failed or returned no records. Aborting SQL Server insertion.")
        return

    # Fetch existing customer IDs from SQL Server
    existing_customer_ids = get_existing_customer_ids()

    # Connect to SQL Server
    print("4. Attempting connection to SQL Server...")
    conn = None
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()
        print("  * Successfully connected to SQL Server.")
        batch_data = []
        orders_with_missing_customer = []
        print("5. Starting to insert/update order data into SQL Server...")

        processed_count = 0
        skipped_missing_customer = 0
        inserted_updated_count = 0

        for order in orders:
            processed_count += 1
            order_id = order.get('id')
            order_barcode = order.get('order_barcode')
            order_date_str = order.get('date_order')
            order_date = datetime.strptime(order_date_str, '%Y-%m-%d %H:%M:%S') if order_date_str else None
            emp_id = order.get('user_id', (False,))[0] if order.get('user_id') else None
            odoo_cust_id = order.get('partner_id', (False,))[0] if order.get('partner_id') else None
            branch_id = order.get('crm_team_id', (False,))[0] if order.get('crm_team_id') else None
            total_bill = order.get('amount_total', 0.0)
            return_status_odoo = order.get('state') # Keeping the Odoo state for now
            return_status = "Not Returned"
            if total_bill < 0:
                return_status = "Returned"

            if all([order_barcode, order_date, total_bill]):
                sql_cust_id = odoo_cust_id if odoo_cust_id in existing_customer_ids else None

                if sql_cust_id is None and odoo_cust_id is not False and odoo_cust_id is not None:
                    orders_with_missing_customer.append({
                        'order_barcode': order_barcode,
                        'odoo_customer_id': odoo_cust_id,
                        'total_bill': total_bill  # Optionally include total_bill
                    })
                    skipped_missing_customer += 1
                    continue

                row = (order_barcode, order_id, order_date, emp_id, sql_cust_id, branch_id, str(total_bill), return_status)
                batch_data.append(row)

                if len(batch_data) >= batch_size:
                    try:
                        cursor.executemany("""
                            MERGE INTO Orders AS target
                            USING (SELECT %(order_barcode)s AS order_barcode, %(order_id)s AS order_id, %(order_date)s AS order_date,
                                         %(emp_id)s AS emp_id, %(cust_id)s AS cust_id, %(branch_id)s AS branch_id,
                                         %(total_bill)s AS total_bill, %(return_status)s AS return_status) AS source
                            ON target.order_barcode = source.order_barcode
                            WHEN MATCHED THEN
                                UPDATE SET
                                    target.order_id = source.order_id,
                                    target.order_date = source.order_date,
                                    target.emp_id = source.emp_id,
                                    target.cust_id = source.cust_id,
                                    target.branch_id = source.branch_id,
                                    target.total_bill = source.total_bill,
                                    target.return_status = source.return_status
                            WHEN NOT MATCHED THEN
                                INSERT (order_id, order_barcode, order_date, emp_id, cust_id, branch_id, total_bill, return_status)
                                VALUES (source.order_id, source.order_barcode, source.order_date, source.emp_id, source.cust_id, source.branch_id, source.total_bill, source.return_status);
                        """, [dict(zip(['order_barcode', 'order_id', 'order_date', 'emp_id', 'cust_id', 'branch_id', 'total_bill', 'return_status'], data)) for data in batch_data])
                        conn.commit()
                        inserted_updated_count += len(batch_data)
                        batch_data = []
                    except pymssql.Error as e:
                        print(f"  * SQL error during batch insertion: {e}")
                        conn.rollback()

        if batch_data:
            try:
                cursor.executemany("""
                    MERGE INTO Orders AS target
                    USING (SELECT %(order_barcode)s AS order_barcode, %(order_id)s AS order_id, %(order_date)s AS order_date,
                                 %(emp_id)s AS emp_id, %(cust_id)s AS cust_id, %(branch_id)s AS branch_id,
                                 %(total_bill)s AS total_bill, %(return_status)s AS return_status) AS source
                    ON target.order_barcode = source.order_barcode
                    WHEN MATCHED THEN
                        UPDATE SET
                            target.order_id = source.order_id,
                            target.order_date = source.order_date,
                            target.emp_id = source.emp_id,
                            target.cust_id = source.cust_id,
                            target.branch_id = source.branch_id,
                            target.total_bill = source.total_bill,
                            target.return_status = source.return_status
                    WHEN NOT MATCHED THEN
                        INSERT (order_id, order_barcode, order_date, emp_id, cust_id, branch_id, total_bill, return_status)
                        VALUES (source.order_id, source.order_barcode, source.order_date, source.emp_id, source.cust_id, source.branch_id, source.total_bill, source.return_status);
                """, [dict(zip(['order_barcode', 'order_id', 'order_date', 'emp_id', 'cust_id', 'branch_id', 'total_bill', 'return_status'], data)) for data in batch_data])
                conn.commit()
                inserted_updated_count += len(batch_data)
            except pymssql.Error as e:
                print(f"  * SQL error during final batch insertion: {e}")
                conn.rollback()

        print("6. Finished inserting/updating order data into SQL Server.")
        print(f"    * Processed {processed_count} order records.")
        print(f"    * Inserted/updated {inserted_updated_count} order records.")
        if orders_with_missing_customer:
            print(f"    * Found {skipped_missing_customer} orders with customer IDs not in the Customers table.")
            write_skipped_orders_to_excel(orders_with_missing_customer, skipped_orders_filename)
        else:
            print("    * No orders skipped due to missing customer IDs.")

        cursor.close()
        conn.close()
        print("  * SQL Server connection closed.")

    except pymssql.Error as e:
        print(f"4. SQL connection error: {e}")
    finally:
        print("7. Order data synchronization process completed.")

# To run the order synchronization, uncomment this line:
process_and_insert_orders()

1. Starting order data synchronization process...
  * Attempting Odoo authentication...
  * Odoo authentication successful. User ID: 112
2. Fetching POS order data from Odoo...
  * Fetching data for model 'pos.order'...
  * Successfully fetched 275420 records for model 'pos.order'.
3. Fetching existing customer IDs from SQL Server...
    * Fetched 841 existing customer IDs.
4. Attempting connection to SQL Server...
  * Successfully connected to SQL Server.
5. Starting to insert/update order data into SQL Server...


## 7-order_details

In [10]:
import xmlrpc.client
import pymssql

# Custom transport with timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'DESKTOP-GMT4735\\SQLEXPRESS'
database_name = 'Pharmacy Dataset 3.0'
sql_user_name = 'sa'
sql_password = 'Mm@12345678'

def sync_order_details():
    print("1. Starting order line synchronization into 'Order_Details'...")

    # Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("  * Odoo authentication failed.")
            return
        print(f"  * Authenticated with Odoo. UID: {uid}")
    except Exception as e:
        print(f"  * Authentication error: {e}")
        return

    # Fetch pos.order.line
    print("2. Fetching order lines from Odoo...")
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        order_lines = models.execute_kw(
            db_name,
            uid,
            password,
            'pos.order.line',
            'search_read',
            [[]],
            {'fields': ['order_id', 'product_id', 'qty']}
        )
        print(f"  * Retrieved {len(order_lines)} lines.")
    except Exception as e:
        print(f"  * Error fetching order lines: {e}")
        return

    # Map order_id to barcode
    order_ids = list(set(line['order_id'][0] for line in order_lines if line['order_id']))
    print("3. Fetching order barcodes...")
    try:
        order_data = models.execute_kw(
            db_name,
            uid,
            password,
            'pos.order',
            'read',
            [order_ids],
            {'fields': ['id', 'order_barcode']}
        )
        order_id_to_barcode = {order['id']: order['order_barcode'] for order in order_data}
    except Exception as e:
        print(f"  * Error fetching barcodes: {e}")
        return

    # Prepare data for insertion
    new_records = []
    for line in order_lines:
        order_id = line.get('order_id', [None])[0]
        product_id = line.get('product_id', [None])[0]
        quantity = line.get('qty', 0.0)
        order_barcode = order_id_to_barcode.get(order_id)

        if order_barcode and product_id:
            new_records.append((order_barcode, product_id, quantity))

    if not new_records:
        print("  * No valid records to insert.")
        return

    # Connect to SQL Server
    print("4. Connecting to SQL Server...")
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        print("5. Fetching existing records to avoid duplicates...")
        cursor.execute("SELECT order_barcode, product_id, quantity FROM Order_Details")
        existing_set = set(cursor.fetchall())

        # Filter out duplicates
        records_to_insert = [rec for rec in new_records if rec not in existing_set]
        print(f"  * Found {len(records_to_insert)} new records to insert (after removing duplicates).")

        if records_to_insert:
            cursor.executemany("""
                INSERT INTO Order_Details (order_barcode, product_id, quantity)
                VALUES (%s, %d, %s)
            """, records_to_insert)
            conn.commit()
            print(f"  * Inserted {len(records_to_insert)} records into 'Order_Details'.")
        else:
            print("  * No new unique records to insert.")

        cursor.close()
        conn.close()
        print("6. SQL Server connection closed.")

    except pymssql.Error as e:
        print(f"  * SQL Server error: {e}")

    print("7. Order line synchronization completed.")

# Run the sync
sync_order_details()


1. Starting order line synchronization into 'Order_Details'...
  * Authenticated with Odoo. UID: 112
2. Fetching order lines from Odoo...
  * Retrieved 418446 lines.
3. Fetching order barcodes...
4. Connecting to SQL Server...
5. Fetching existing records to avoid duplicates...
  * Found 418446 new records to insert (after removing duplicates).
  * Inserted 418446 records into 'Order_Details'.
6. SQL Server connection closed.
7. Order line synchronization completed.


## 8-Payments

In [ ]:
import xmlrpc.client
import pymssql
import pandas as pd
import time

class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=600):  # Increased timeout to 600 seconds (10 minutes)
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo connection details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# Unified SQL Server connection
def get_sql_connection():
    return pymssql.connect(
        server='DESKTOP-GMT4735\\SQLEXPRESS',
        user='sa',
        password='Mm@12345678',
        database='Pharmacy Dataset 3.0',
        charset='utf8'
    )

# Fetch data in batches with retry logic
def fetch_data_in_batches(model, fields, uid, batch_size=500):  # Reduced batch size to 500
    all_data = []
    offset = 0

    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=600))
        while True:
            batch = models.execute_kw(
                db_name, uid, password, model, 'search_read',
                [[]],
                {'fields': fields, 'limit': batch_size, 'offset': offset}
            )

            if not batch:
                break

            all_data.extend(batch)
            offset += batch_size
            print(f"Fetched batch: {offset}")

        return all_data

    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def fetch_data_with_retry(model, fields, uid, batch_size=500, retries=3, delay=5):
    attempt = 0
    while attempt < retries:
        try:
            return fetch_data_in_batches(model, fields, uid, batch_size)
        except Exception as e:
            attempt += 1
            print(f"Error fetching data (attempt {attempt}): {e}")
            if attempt < retries:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("Max retries reached. Could not fetch data.")
                return []

def process_payments():
    print("Starting data synchronization for Payments...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=600))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch POS payments in batches
    print("Fetching POS payments...")
    payments = fetch_data_with_retry('pos.payment', ['id', 'pos_order_id', 'amount', 'payment_method_id'], uid, batch_size=500)
    print(f"Finished fetching {len(payments)} payment records.")

    if not payments:
        print("No payments fetched.")
        return

    # Step 3: Connect to SQL Server
    try:
        conn = get_sql_connection()
        cursor = conn.cursor()
        skipped = []

        for p in payments:
            payment_id = p.get('id')
            order_info = p.get('pos_order_id')
            method_info = p.get('payment_method_id')
            order_barcode = order_info[1] if isinstance(order_info, list) and len(order_info) > 1 else None
            payment_method = method_info[1] if isinstance(method_info, list) and len(method_info) > 1 else None
            payment_amount = p.get('amount', 0.0)

            # Check if order_barcode exists in Orders table
            cursor.execute("SELECT COUNT(*) FROM Orders WHERE order_barcode = %s", (order_barcode,))
            exists = cursor.fetchone()[0]

            if exists:
                try:
                    cursor.execute("""
                        INSERT INTO Payments (payment_id, order_barcode, payment_amount, payment_method)
                        VALUES (%s, %s, %s, %s)
                    """, (payment_id, order_barcode, payment_amount, payment_method))
                except pymssql.Error as e:
                    skipped.append({
                        'payment_id': payment_id,
                        'order_barcode': order_barcode,
                        'payment_amount': payment_amount,
                        'payment_method': payment_method,
                        'error': str(e)
                    })
            else:
                skipped.append({
                    'payment_id': payment_id,
                    'order_barcode': order_barcode,
                    'payment_amount': payment_amount,
                    'payment_method': payment_method,
                    'error': 'Order barcode not found in Orders table'
                })

        conn.commit()
        print("Finished inserting all valid payments.")

        if skipped:
            df = pd.DataFrame(skipped)
            df.to_excel("skipped_payments.xlsx", index=False)
            print(f"Skipped {len(skipped)} payments. Logged to 'skipped_payments.xlsx'.")

        cursor.close()
        conn.close()

    except Exception as e:
        print(f"SQL error: {e}")

# Run the function
process_payments()


Starting data synchronization for Payments...
Authenticated successfully. User ID: 112
Fetching POS payments...
Fetched batch: 500
Fetched batch: 1000
Fetched batch: 1500
Fetched batch: 2000
Fetched batch: 2500
Fetched batch: 3000
Fetched batch: 3500
Fetched batch: 4000
Fetched batch: 4500
Fetched batch: 5000
Fetched batch: 5500
Fetched batch: 6000
Fetched batch: 6500
Fetched batch: 7000
Fetched batch: 7500
Fetched batch: 8000
Fetched batch: 8500
Fetched batch: 9000
Fetched batch: 9500
Fetched batch: 10000
Fetched batch: 10500
Fetched batch: 11000
Fetched batch: 11500
Fetched batch: 12000
Fetched batch: 12500
Fetched batch: 13000
Fetched batch: 13500
Fetched batch: 14000
Fetched batch: 14500
Fetched batch: 15000
Fetched batch: 15500
Fetched batch: 16000
Fetched batch: 16500
Fetched batch: 17000
Fetched batch: 17500
Fetched batch: 18000
Fetched batch: 18500
Fetched batch: 19000
Fetched batch: 19500
Fetched batch: 20000
Fetched batch: 20500
Fetched batch: 21000
Fetched batch: 21500
Fetch

## 9-Inventory 

In [ ]:
import xmlrpc.client
import pymssql
import pandas as pd

class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo connection details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# Unified SQL Server connection
def get_sql_connection():
    return pymssql.connect(
        server='DESKTOP-GMT4735\\SQLEXPRESS',
        user='sa',
        password='Mm@12345678',
        database='Pharmacy Dataset 2.0',
        charset='utf8'
    )

def fetch_data(model, fields, uid):
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        return models.execute_kw(db_name, uid, password, model, 'search_read', [[]], {'fields': fields})
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def sync_inventory():
    print("Starting inventory sync...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch stock quantities from Odoo
    print("Fetching product stock data...")
    stock_data = fetch_data('product.product', ['id', 'qty_available'], uid)
    print(f"Fetched {len(stock_data)} product records.")

    if not stock_data:
        return

    # Step 3: Connect to SQL Server
    try:
        conn = get_sql_connection()
        cursor = conn.cursor()
        skipped = []

        for item in stock_data:
            product_id = item.get('id')
            stock = item.get('qty_available', 0)

            # Check if product_id exists in Products table
            cursor.execute("SELECT COUNT(*) FROM Products WHERE product_id = %s", (product_id,))
            exists = cursor.fetchone()[0]

            if exists:
                try:
                    cursor.execute("""
                        INSERT INTO Inventory (inventory_product_id, stock)
                        VALUES (%s, %s)
                    """, (product_id, stock))
                except pymssql.Error as e:
                    skipped.append({
                        'product_id': product_id,
                        'stock': stock,
                        'error': str(e)
                    })
            else:
                skipped.append({
                    'product_id': product_id,
                    'stock': stock,
                    'error': 'Product not found in Products table'
                })

        conn.commit()
        print("Finished inserting inventory data.")

        if skipped:
            df = pd.DataFrame(skipped)
            df.to_excel("skipped_inventory.xlsx", index=False)
            print(f"Skipped {len(skipped)} inventory records. Logged to 'skipped_inventory.xlsx'.")

        cursor.close()
        conn.close()

    except Exception as e:
        print(f"SQL error: {e}")

# Run the function
sync_inventory()
